### Histogram 1D

In [1]:
import numpy as np
import time
import asyncio as aio
import pandas as pd
from vega.widget import VegaWidget
nbins = 50
spec_no_data = {
    '$schema': 'https://vega.github.io/schema/vega-lite/v4.8.1.json',
     'config': {'view': {'continuousHeight': 300, 'continuousWidth': 400}},
     'data': {'name': 'data'},
     'encoding': {
                  'x': {'field': 'x', 'type': 'ordinal'},
                  'y': {'field': 'y', 'type': 'quantitative'}
             },
     'mark': 'bar'}
spec_with_data = spec_no_data.copy()
nsamples = 100_000
steps = 1_000
rdata = np.random.normal(loc=0, scale=0.1, size=nsamples)
widget = VegaWidget(spec=spec_no_data)
display(widget)
async def progressive_loop(widget, nbins=50):
    hist = np.zeros(nbins)
    for sl in np.split(rdata, steps):
        await aio.sleep(0.1)
        h, _ = np.histogram(sl, bins=nbins, range=(-1.0,1.0))
        hist += h
        source = pd.DataFrame({
            'x': range(nbins),
            'y': hist,
        }, dtype='float32')
        #  widget.update('data', remove='datum.x>=0')
        #print("displayed", widget._displayed)
        widget._displayed = True
        widget.update_dataframe(source, remove="true")

await progressive_loop(widget)

VegaWidget()

CancelledError: 

### Histogram 2D

In [2]:
import time
import numpy as np
import asyncio as aio
import pandas as pd
from vega.widget import VegaWidget
import ipywidgets as widgets
nbins = 256  # 128
spec_no_data = {'$schema': 'https://vega.github.io/schema/vega-lite/v4.8.1.json',
 "width": 500,
 "height": 400,
 'data': {'name': 'data'},
 'encoding': {'color': {'field': 'z', 
                        'type': 'quantitative',
                        "scale": {"domain": [0,1]}},
  'x': {'field': 'x', 'type': 'ordinal'},
  'y': {'field': 'y', 'type': 'ordinal'}},
 'mark': 'rect',
 "config": {
    "axis": {
      "disable": True
    }
  }}
widget = VegaWidget(spec=spec_no_data)
steps = 100
progress = widgets.IntProgress(value=0, min=0, max=steps, description='progress', orientation='horizontal')
mean = widgets.FloatText(value=0, description='Time mean', disables=True)
std = widgets.FloatText(value=0, description='Time std ', disables=True)
display(widgets.VBox([widget, progress, mean, std]))
nsamples=100_1000
means = [0.1, 0.3]
cov = [[0.01, 0], [0, 0.09]]
rdata = np.random.multivariate_normal(means, cov, size=(nsamples))
async def progressive_loop(widget, nbins=128):
    hist = np.zeros((nbins,nbins), dtype='float32')
    step = 0
    times = []
    for sl in np.split(rdata, steps):
        progress.value = step
        step += 1
        await aio.sleep(0)
        h, *_ = np.histogram2d(sl[:,0],sl[:,1], bins=nbins, range=((-1.0,1.0),(-1.0,1.0)))
        hist += h
        max = np.max(hist)
        #print(hist.shape)
        widget._displayed = True
        now = time.time()
        widget.update_histogram2d(hist/max, ['x', 'y', 'z'], remove="true")
        times.append(time.time() - now)
        mean.value = np.mean(times)
        std.value = np.std(times)

await progressive_loop(widget)